### Decoupled Data Parsers

In [ ]:
import anndata
import os
import requests

save_path = "data/example_sce.h5ad"
if not os.path.exists(save_path):
    response = requests.get("https://go.wisc.edu/69435h")
    with open(save_path, "wb") as f:
        f.write(response.content)

example_sce = anndata.read_h5ad(save_path)
example_sce

In [ ]:
from scdesigner.experimental.data import FormulaLoader

dl = FormulaLoader(example_sce, {"mu": "~ pseudotime", "alpha": "~ 1"}, batch_size=1000)
y, x = next(iter(dl.loader))
print(dl.names)
print(y, x)

### Generic Estimators

In [ ]:
from scdesigner.experimental.estimators import NegativeBinomialML

dl = FormulaLoader(example_sce, {"mu": "~ pseudotime", "alpha": "~ 1"}, batch_size=1000)
ml = NegativeBinomialML({"lr": 0.01, "max_epochs": 10})
parameters = ml.estimate(dl.loader)

In [ ]:
from scdesigner.experimental.samplers import NegativeBinomialSampler

sampler = NegativeBinomialSampler(parameters)
sampler.sample(dl.loader)

The more realistic case is when the loader only has covariate information, not the original training Y gene count assay.

In [ ]:
dl = FormulaLoader(example_sce.obs, {"mu": "~ pseudotime", "alpha": "~ 1"}, batch_size=1000)
sampler.sample(dl.loader)